# LOL game prediction

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras

In [3]:
from google.colab import files
uploaded = files.upload()

Saving lol.csv to lol.csv


In [4]:
df = pd.read_csv("./lol.csv")

df

,gameId,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,...,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
0,4519157822,0,28,2,1,9,6,11,0,0,...,0,16567,6.8,17047,197,55,-643,8,19.7,1656.7
1,4523371949,0,12,1,0,5,5,5,0,0,...,1,17620,6.8,17438,240,52,2908,1173,24.0,1762.0
2,4521474530,0,15,0,0,7,11,4,1,1,...,0,17285,6.8,17254,203,28,1172,1033,20.3,1728.5
3,4524384067,0,43,1,0,4,5,5,1,0,...,0,16478,7.0,17961,235,47,1321,7,23.5,1647.8
4,4436033771,0,75,4,0,6,6,6,0,0,...,0,17404,7.0,18313,225,67,1004,-230,22.5,1740.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9874,4527873286,1,17,2,1,7,4,5,1,1,...,0,15246,6.8,16498,229,34,-2519,-2469,22.9,1524.6
9875,4527797466,1,54,0,0,6,4,8,1,1,...,0,15456,7.0,18367,206,56,-782,-888,20.6,1545.6
9876,4527713716,0,23,1,0,6,7,5,0,0,...,0,18319,7.4,19909,261,60,2416,1877,26.1,1831.9
9877,4527628313,0,14,4,1,2,3,3,1,1,...,0,15298,7.2,18314,247,40,839,1085,24.7,1529.8


In [5]:
ys_columns = ['blueWins']
xs_columns = [
       'blueWardsPlaced', 'blueWardsDestroyed',
       'blueFirstBlood', 'blueKills', 'blueDeaths', 'blueAssists',
       'blueEliteMonsters', 'blueDragons', 'blueHeralds', 'blueTowersDestroyed', 
       'blueTotalGold', 'blueAvgLevel', 'blueTotalExperience', 
       'blueTotalMinionsKilled', 'blueTotalJungleMinionsKilled', 'blueGoldDiff', 
       'blueExperienceDiff','blueCSPerMin', 'blueGoldPerMin',
                     
       'redWardsPlaced', 'redWardsDestroyed',
       'redFirstBlood', 'redKills', 'redDeaths', 'redAssists',
       'redEliteMonsters', 'redDragons', 'redHeralds', 'redTowersDestroyed',
       'redTotalGold', 'redAvgLevel', 'redTotalExperience',
       'redTotalMinionsKilled', 'redTotalJungleMinionsKilled', 'redGoldDiff',
       'redExperienceDiff', 'redCSPerMin', 'redGoldPerMin'
]

In [6]:
xs = df.loc[:, xs_columns]
ys = df.loc[:, ys_columns]

In [ ]:
# Data exploration
# leave here for reference

ncols, nrows = 5, 8
plt.figure(figsize=(ncols*3,nrows*3))
for i, column in enumerate(xs.columns):
    plt.subplot(nrows, ncols, i+1)
    plt.hist(xs.loc[:,column], bins=32)
    plt.title(column)
plt.tight_layout()

In [11]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df)

print(f"df_train.shape = {df_train.shape}")
print(f"df_test.shape = {df_test.shape}")

df_train.shape = (7409, 40)
df_test.shape = (2470, 40)


In [12]:
ys_columns = ['blueWins']
xs_columns = [
    'TotalGold', 'GoldDiff', 'GoldPerMin',
    'TotalExperience','ExperienceDiff','AvgLevel',
    'Kills', 'Deaths', 'Assists','FirstBlood',
    'WardsPlaced', 'WardsDestroyed',
    'TotalMinionsKilled','TotalJungleMinionsKilled',
    'EliteMonsters', 'Dragons', 'Heralds',
    'TowersDestroyed', 'CSPerMin'
]

xs_blue_columns = ['blue'+column for column in xs_columns]
xs_red_columns = ['red'+column for column in xs_columns]

In [ ]:
xs_train = df_train[xs_blue_columns+xs_red_columns]
xs_test = df_test[xs_blue_columns+xs_red_columns]

ys_train = df_train[ys_columns]
ys_test = df_test[ys_columns]

In [13]:
# Data augmentation step to improve the model performance
# for reference

def red_blue_symmetry(df):
    xs_blue = df.loc[:, xs_blue_columns].values
    xs_red = df.loc[:, xs_red_columns].values
    ys_blue = df.loc[:, ys_columns].values[:,0]
    ys_red = 1-ys_blue

    xs = np.vstack([
        np.hstack([xs_blue,xs_red]),
        np.hstack([xs_red,xs_blue])
    ])
    
    ys = np.hstack([
        ys_blue,
        ys_red
    ])

    return xs, ys

In [14]:
xs_train, ys_train = red_blue_symmetry(df_train)
xs_test, ys_test = red_blue_symmetry(df_test)

print(f"xs_train.shape = {xs_train.shape}")
print(f"ys_train.shape = {ys_train.shape}")
print()
print(f"xs_test.shape = {xs_test.shape}")
print(f"ys_test.shape = {ys_test.shape}")

xs_train.shape = (14818, 38)
ys_train.shape = (14818,)

xs_test.shape = (4940, 38)
ys_test.shape = (4940,)


In [15]:
xs_train = df_train[xs_blue_columns+xs_red_columns]
xs_test = df_test[xs_blue_columns+xs_red_columns]

ys_train = df_train[ys_columns]
ys_test = df_test[ys_columns]

***Neural Network***

In [22]:
model = keras.Sequential(
    [
     keras.layers.Dense(16, activation='tanh'),
     keras.layers.Dense(32, activation='tanh'),
     keras.layers.Dense(32, activation='tanh'),
     keras.layers.Dense(1),
     keras.layers.Lambda(lambda x: tf.sigmoid(x)),
])
model

In [24]:
model.compile(
              optimizer=keras.optimizers.Adam(1e-4),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=["accuracy"]
)
history=model.fit(xs_train, ys_train, validation_data=(xs_test, ys_test), epochs=10)
history

Epoch 1/10
232/232 [==============================] - 2s 3ms/step - loss: 0.5517 - accuracy: 0.7240 - val_loss: 0.5620 - val_accuracy: 0.7186
Epoch 2/10
232/232 [==============================] - 1s 3ms/step - loss: 0.5500 - accuracy: 0.7270 - val_loss: 0.5600 - val_accuracy: 0.7198
Epoch 3/10
232/232 [==============================] - 1s 3ms/step - loss: 0.5529 - accuracy: 0.7255 - val_loss: 0.5591 - val_accuracy: 0.7235
Epoch 4/10
232/232 [==============================] - 1s 3ms/step - loss: 0.5549 - accuracy: 0.7218 - val_loss: 0.5642 - val_accuracy: 0.7194
Epoch 5/10
232/232 [==============================] - 1s 4ms/step - loss: 0.5540 - accuracy: 0.7260 - val_loss: 0.5605 - val_accuracy: 0.7166
Epoch 6/10
232/232 [==============================] - 1s 5ms/step - loss: 0.5566 - accuracy: 0.7244 - val_loss: 0.5603 - val_accuracy: 0.7146
Epoch 7/10
232/232 [==============================] - 1s 4ms/step - loss: 0.5561 - accuracy: 0.7247 - val_loss: 0.5645 - val_accuracy: 0.7117
Epoch 

In [25]:
test_loss, test_acc = model.evaluate(xs_test, ys_test)
print(test_acc)

78/78 [==============================] - 0s 1ms/step - loss: 0.5602 - accuracy: 0.7170
0.7170040607452393
